In [10]:
import os
from dotenv import load_dotenv
load_dotenv()

# Imports from the Google API client libraries
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

# LangChain imports
from langchain_core.prompts import PromptTemplate
from langchain_classic.chains import LLMChain
from langchain_google_genai import ChatGoogleGenerativeAI



In [11]:
# Gmail API Configuration
# Define the scopes and file paths for credentials and token
# If modifying these SCOPES, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/gmail.modify']
CREDENTIALS_FILE = '../credentials/credentials.json'
TOKEN_FILE = 'token.json'

In [12]:
# ================================
# Improved Email Classification Prompt
# ================================

EMAIL_CLASSIFICATION_PROMPT = """
You are a precise email classifier for job-related emails.
Your task is to assign the email to EXACTLY ONE of the following category keys:

{categories}

=========================
CATEGORY DEFINITIONS
=========================

application_confirmed:
    Confirmation that an application was received.

interview_request:
    Recruiter requesting to schedule an interview or providing interview scheduling links.

interview_reminder:
    Reminders or confirmations for upcoming interviews.

offer:
    Job offer emails, offer letters, verbal offers, or negotiation instructions.

rejected:
    Rejection emails stating the applicant is not moving forward.

assessment:
    Coding tests, online assessments, take-home assignments.

follow_up:
    Recruiter checking in, following up, asking for updates, or next-steps clarification.

job_alert:
    Job recommendations, alerts about openings, job board notifications.

newsletter:
    Company newsletters, weekly digests, marketing content.

spam:
    Irrelevant, suspicious, non-job content.

uncategorized:
    Use ONLY if the email clearly does not fit any category above.

=========================
CATEGORY PRIORITY
=========================
interview_request > interview_reminder > offer > rejected > assessment >
follow_up > application_confirmed > job_alert > newsletter > spam > uncategorized

=========================
EMAIL TO CLASSIFY
=========================

Subject: {subject}
Snippet: {snippet}

=========================
RESPONSE RULES
=========================
- Respond with ONLY the category key.
- No explanation.
- No extra text.
- No formatting.
Category:
"""


In [13]:
JOB_CATEGORIES = {
    "application_confirmed": {"label": "Applied ✓"},
    "interview_request": {"label": "Interview 📅"},
    "interview_reminder": {"label": "Interview Reminder ⏰"},
    "offer": {"label": "Job Offer 🎉"},
    "rejected": {"label": "Rejected ❌"},
    "assessment": {"label": "Assessment 📝"},
    "follow_up": {"label": "Follow-up 💬"},
    "job_alert": {"label": "Job Alert 🔔"},
    "newsletter": {"label": "Newsletter 📰"},
    "spam": {"label": "Spam 🗑️"},
    "uncategorized": {"label": "Other 📧"}
}


In [18]:
class GmailHandler:
    """Minimal Gmail fetcher
    Fetch the labels and unread emails from Gmail using Gmail API.
    """

    def __init__(self):
        self.service = None
        self.authenticate()

    def authenticate(self):
        """Authenticate with Gmail API"""
        creds = None

        # Load existing token
        if os.path.exists(TOKEN_FILE):
            creds = Credentials.from_authorized_user_file(TOKEN_FILE, SCOPES)
            print("creds", creds.to_json())

        # If no valid token, log in
        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(CREDENTIALS_FILE, SCOPES)
                creds = flow.run_local_server(port=0)
            # Save token
            with open(TOKEN_FILE, 'w') as token:
                token.write(creds.to_json())

        self.service = build('gmail', 'v1', credentials=creds)

    def get_labels(self):
        """Fetch and display only user-created Gmail labels"""
        try:
            results = self.service.users().labels().list(userId='me').execute()
            labels = results.get('labels', [])

            # Filter only user-created labels
            user_labels = [label for label in labels if label.get('type') == 'user']

            if not user_labels:
                print("⚠️ No user-created labels found.")
                return []

            print("🏷️  Your Gmail Labels (created by you):")
            for label in user_labels:
                print(f" - {label['name']}")
            print()
            return user_labels

        except Exception as e:
            print(f"❌ Error fetching labels: {e}")
            return []
        
    def get_email_details(self, msg_id):
        """Get full email details + labels"""
        try:
            msg = self.service.users().messages().get(
                userId='me', id=msg_id, format='full'
            ).execute()

            headers = msg['payload']['headers']
            subject = next((h['value'] for h in headers if h['name'] == 'Subject'), '(No Subject)')
            sender = next((h['value'] for h in headers if h['name'] == 'From'), '(Unknown)')
            snippet = msg.get('snippet', '')
            labels = msg.get('labelIds', [])

            print(f"📧 From: {sender}")
            print(f"   Subject: {subject}")
            print(f"   Labels: {', '.join(labels) if labels else '(No Labels)'}")
            print(f"   Snippet: {snippet}...\n")

            return {"id": msg_id, "from": sender, "subject": subject, "labels": labels, "snippet": snippet}

        except Exception as e:
            print(f"❌ Error reading email: {e}")
            return None

    def get_emails_since(self, hours: int = 24, max_results: int = 50, unread_only: bool = False):
        """Fetch emails received in the last 'hours' hours"""
        try:
            if unread_only:
                query = f"newer_than:{hours}h is:unread"
            else:
                query = f"newer_than:{hours}h"
            results = self.service.users().messages().list(
                userId='me',
                labelIds=['INBOX'],
                q=query,
                maxResults=max_results
            ).execute()

            messages = results.get('messages', [])
            if not messages:
                print(f"✅ No emails found in the last {hours} hours.")
                return []

            print(f"📬 Found {len(messages)} emails from the last {hours} hours:\n")
            emails = []
            for msg in messages:
                details = self.get_email_details(msg['id'])
                if details:
                    emails.append(details)
            return emails

        except Exception as e:
            print(f"❌ Error fetching emails: {e}")
            return []

    def check_label_exists(self, label_name):
        """Check if a label exists"""
        labels = self.get_labels()
        for label in labels:
            if label['name'].lower() == label_name.lower():
                return True
        return False
    
    def create_label(self, label_name):
        """Create a new label"""
        label_body = {
            'name': label_name,
            'labelListVisibility': 'labelShow',
            'messageListVisibility': 'show'
        }
        try:
            label = self.service.users().labels().create(
                userId='me',
                body=label_body
            ).execute()
            print(f"✅ Label '{label_name}' created.")
            return label
        except Exception as e:
            print(f"❌ Error creating label: {e}")
            return None
        
    def migrate_old_labels(self, old_to_new_map: dict):
        """
        Move emails from old labels to new labels, then delete the old labels.
        
        old_to_new_map: dict mapping old label names -> new label names
            e.g., {"Ghosted": "Follow-up 💬", "Rejection": "Rejected ❌"}
        """
        for old_label, new_label in old_to_new_map.items():
            print(f"\n📦 Migrating emails from '{old_label}' → '{new_label}'")
            
            # 1️⃣ Get old label ID
            labels = self.get_labels()
            old_label_obj = next((l for l in labels if l['name'].lower() == old_label.lower()), None)
            if not old_label_obj:
                print(f"⚠️ Old label '{old_label}' not found, skipping.")
                continue
            old_label_id = old_label_obj['id']

            # 2️⃣ Ensure new label exists
            if not self.check_label_exists(new_label):
                self.create_label(new_label)
            new_label_id = next((l['id'] for l in self.get_labels() if l['name'].lower() == new_label.lower()), None)

            # 3️⃣ Fetch emails with old label
            try:
                results = self.service.users().messages().list(
                    userId='me',
                    labelIds=[old_label_id]
                ).execute()
                messages = results.get('messages', [])
                if not messages:
                    print(f"✅ No emails found under '{old_label}'")
                    continue

                print(f"📬 Found {len(messages)} emails under '{old_label}'")

                # 4️⃣ Move emails to new label
                for msg in messages:
                    self.service.users().messages().modify(
                        userId='me',
                        id=msg['id'],
                        body={'addLabelIds': [new_label_id], 'removeLabelIds': [old_label_id]}
                    ).execute()
                print(f"✅ Migrated {len(messages)} emails to '{new_label}'")

            except Exception as e:
                print(f"❌ Error migrating emails from '{old_label}': {e}")

            # 5️⃣ Delete old label
            try:
                self.service.users().labels().delete(userId='me', id=old_label_id).execute()
                print(f"🗑️ Deleted old label '{old_label}'")
            except Exception as e:
                print(f"❌ Error deleting old label '{old_label}': {e}")



In [19]:
class EmailCategorizer:
    """Email categorizer using LLM Model defined by user."""
    
    def __init__(self, llm):
        self.llm = llm
        self.prompt_template = PromptTemplate(
            input_variables=["categories", "subject", "snippet"],
            template=EMAIL_CLASSIFICATION_PROMPT
        )
        self.chain = LLMChain(llm=self.llm, prompt=self.prompt_template)


    def categorize(self, emails):
        """Categorize a list of emails"""
        results = []
        categories_str = ", ".join(JOB_CATEGORIES.keys())

        for email in emails:
            subject = email.get("subject", "")
            snippet = email.get("snippet", "")
            try:
                response = self.chain.run(
                    subject=subject,
                    snippet=snippet,
                    categories=categories_str
                ).strip().lower()

                # match valid category or fallback
                category_key = (
                    response if response in JOB_CATEGORIES else "uncategorized"
                )
                label = JOB_CATEGORIES[category_key]["label"]

                email["category"] = category_key
                email["category_label"] = label
                results.append(email)

                print(f"📧 {subject[:60]} → {label}")

            except Exception as e:
                print(f"❌ Error classifying email '{subject}': {e}")

        return results


In [20]:
class GmailLLMAgent:
    """
    Fetch emails from Gmail Classifier, classify them using LLM Class, 
    and apply labels based on the predicted category and update it in Gmail.

    """

    def __init__(self, gmail_handler: GmailHandler, categorizer: EmailCategorizer):
        self.gmail = gmail_handler
        self.categorizer = categorizer

    def process_emails(self, hours: int = 24, max_results: int = 5, unread_only: bool = False):
        """
        Fetch emails from the last 'hours' hours, classify, and label them.
        """
        emails = self.gmail.get_emails_since(hours=hours, max_results=max_results, unread_only=unread_only)
        if not emails:
            print("📭 No emails to process.")
            return

        print(f"📬 Processing {len(emails)} emails...\n")

        # 1️⃣ Classify emails
        categorized_emails = self.categorizer.categorize(emails)

        # 2️⃣ Apply labels
        for email in categorized_emails:
            label_name = email.get("category_label", "Other 📧")

            # create label if not exists
            if not self.gmail.check_label_exists(label_name):
                self.gmail.create_label(label_name)

            # apply label
            try:
                self.gmail.service.users().messages().modify(
                    userId='me',
                    id=email['id'],
                    body={'addLabelIds': [self._get_label_id(label_name)],
                          'removeLabelIds': []}  # optional: remove UNREAD
                ).execute()
                print(f"✅ Labeled: {email['subject'][:50]} → {label_name}")
            except Exception as e:
                print(f"❌ Error applying label to '{email['subject']}': {e}")

    def _get_label_id(self, label_name: str):
        """Return Gmail label ID, create if it doesn't exist"""
        labels = self.gmail.get_labels()
        for label in labels:
            if label['name'].lower() == label_name.lower():
                return label['id']
        # create if not found
        label = self.gmail.create_label(label_name)
        return label['id'] if label else None


In [21]:


# Initialize Gmail handler
gmail_handler = GmailHandler()

# Initialize Groq email categorizer
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

llm = ChatGoogleGenerativeAI(google_api_key=GOOGLE_API_KEY,
            model="gemini-2.5-flash")

categorizer = EmailCategorizer(llm)

# Create LLM Agent
agent = GmailLLMAgent(gmail_handler, categorizer)

# Process emails from last 24 hours
agent.process_emails(hours=24, max_results=20, unread_only=True)


creds {"token": "ya29.a0ATi6K2ubRD8MBUiYzJQfOjpF7LWYbimoz9Eegucry4TT0zcXoen7PYLEcGTwzsA9PIOCqXiwJdD3ZfweNIv5i6RmLC9FQvYURY9_b__NfvkziM1gZ6BUQGRJXIf5D6iJaM2X6xvteQ8yWCuVtRSZzShPu9C1wRWer2OEDAXJsqwBDBLnZfzIErEFD9H4bi5np-bOJhXNaCgYKAf0SARMSFQHGX2Mi3i1GTTUcoUR8XB_d5Xp-Zg0207", "refresh_token": "1//05TrQi9MqXxoTCgYIARAAGAUSNwF-L9IrtueZ2vud33vXcr9aQhXc0R1T7ekq0Lv2GFgN_cvw8k0yDL-GLPc46lwwoOOwCdir4lc", "token_uri": "https://oauth2.googleapis.com/token", "client_id": "643813721017-2err0bl1c4ueihcvsgbqm7cckvulsip6.apps.googleusercontent.com", "client_secret": "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", "scopes": ["https://www.googleapis.com/auth/gmail.modify"], "universe_domain": "googleapis.com", "account": "", "expiry": "2025-11-18T01:31:10Z"}


C:\Users\Abhishek\AppData\Local\Temp\ipykernel_1928\1668750096.py:10: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use `RunnableSequence, e.g., `prompt | llm`` instead.
  self.chain = LLMChain(llm=self.llm, prompt=self.prompt_template)


📬 Found 20 emails from the last 24 hours:

📧 From: TikTok <notification@service.tiktok.com>
   Subject: kingdelontae posted: I wasn't going to post  this but dance is the onl...
   Labels: CATEGORY_PROMOTIONS, UNREAD, INBOX
   Snippet: Hot in #Dance: 87.4K+ likes and counting ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌...

📧 From: Glassdoor Community <noreply@glassdoor.com>
   Subject: The latest buzz on ZS Associates
   Labels: UNREAD, CATEGORY_UPDATES, INBOX
   Snippet: Hey Abhishek, catch up on recent conversations ‌​‍‎‏﻿ ‌​‍‎‏﻿ ‌​‍‎‏﻿ ‌​‍‎‏﻿ ‌​‍‎‏﻿ ‌​‍‎‏﻿ ‌​‍‎‏﻿ ‌​‍‎‏﻿ ‌​‍‎‏﻿ ‌​‍‎‏﻿ ‌​‍‎‏﻿ ‌​‍‎‏﻿ ‌​‍‎‏﻿ ‌​‍‎‏﻿ ‌​‍‎‏﻿ ‌​‍‎‏﻿ ‌​‍‎‏﻿ ‌​‍‎‏﻿ ‌​‍‎‏﻿ ‌​‍‎‏﻿ ‌​‍‎‏﻿ ‌​‍‎‏﻿...

📧 From: FreshCo <freshco@em.freshco.com>
   Subject: Save on fresh blooms, sweet treats & bulk baking for the holidays 🎄
   Labels: CATEGORY_PROMOTIONS, UNREAD, INBOX
   Snippet: Download the FreshCo ap

C:\Users\Abhishek\AppData\Local\Temp\ipykernel_1928\1668750096.py:22: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  response = self.chain.run(


📧 kingdelontae posted: I wasn't going to post  this but dance  → Newsletter 📰
📧 The latest buzz on ZS Associates → Newsletter 📰
📧 Save on fresh blooms, sweet treats & bulk baking for the hol → Newsletter 📰
📧 Make your AI agents faster and more cost-effective with sema → Newsletter 📰
📧 Just Launched: Gemini 3 Pro Takes the Lead on Multimodal Rea → Newsletter 📰
📧 Ragini pandey🧿🧿🧿🧿🧿😍😍😍 and others posted 3 videos → Spam 🗑️
📧 Full Stack Software Developer at TELUS and 10 more jobs in T → Job Alert 🔔
📧 Founding Engineer - Full Stack at mLabs (India) and 7 more j → Job Alert 🔔
📧 Violet Green  posted in Microsoft Reactor Toronto → Job Alert 🔔


...

📧 “Canada”: Toronto Paramedic Services (Formerly Toronto EMS)  → Job Alert 🔔


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 10, model: gemini-2.5-flash
Please retry in 52.399937342s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 1

📧 Data Analyst role at Blackhawk Network: you would be a great → Job Alert 🔔


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 10, model: gemini-2.5-flash
Please retry in 50.278725155s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 1

📧 Senior Software Engineering Manager at Capital One and 7 mor → Job Alert 🔔
📧 📅 Just scheduled: Get started with AI agent development → Newsletter 📰
📧 My ex broke up with me 9 months ago because he fell out of l → Spam 🗑️
📧 ✈ Cheap Round Trip Deals! → Spam 🗑️
📧 [Remote] Mercury - Customer Support → Job Alert 🔔
📧 Work in Ottawa: one new job matching your profile → Job Alert 🔔
📧 New jobs in Canada. Apply Now. → Job Alert 🔔
📧 BetterMe, remote: Growth Product Manager (WEB) → Job Alert 🔔
📧 Co-op General at Nova Scotia Power and 3 more jobs in Canada → Job Alert 🔔
🏷️  Your Gmail Labels (created by you):
 - Newsletter 📰
 - Interview 📅
 - Job Alert 🔔
 - Applied ✓
 - Follow-up 💬
 - Rejected ❌
 - Other 📧
 - Spam 🗑️

🏷️  Your Gmail Labels (created by you):
 - Newsletter 📰
 - Interview 📅
 - Job Alert 🔔
 - Applied ✓
 - Follow-up 💬
 - Rejected ❌
 - Other 📧
 - Spam 🗑️

✅ Labeled: kingdelontae posted: I wasn't going to post  this  → Newsletter 📰
🏷️  Your Gmail Labels (created by you):
 - Newsletter 📰